In [1]:
import urllib
import random
import re
import nltk
import string
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PCZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PCZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PCZ\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pandas as pd

In [287]:
auth1_url1 = "https://www.gutenberg.org/files/786/786-0.txt"
auth1_url2 = "https://www.gutenberg.org/files/564/564-0.txt"
auth1_url3 = "https://www.gutenberg.org/files/675/675-0.txt"
auth1_url4 = "https://www.gutenberg.org/files/98/98-0.txt"
auth1_url5 = "https://www.gutenberg.org/files/650/650-0.txt"

In [288]:
auth2_url1 = "https://www.gutenberg.org/files/141/141-0.txt"
auth2_url2 = "https://www.gutenberg.org/cache/epub/21839/pg21839.txt"
auth2_url3 = "https://www.gutenberg.org/cache/epub/42671/pg42671.txt"
auth2_url4 = "https://www.gutenberg.org/cache/epub/105/pg105.txt"
auth2_url5 = "https://www.gutenberg.org/files/121/121-0.txt"

In [289]:
auth3_url1 = "https://www.gutenberg.org/files/108/108-0.txt"
auth3_url2 = "https://www.gutenberg.org/files/834/834-0.txt"
auth3_url3 = "https://www.gutenberg.org/files/903/903-0.txt"
auth3_url4 = "https://www.gutenberg.org/files/48320/48320-0.txt"
auth3_url5 = "https://www.gutenberg.org/files/54109/54109-0.txt"

In [290]:
auth4_url1 = "https://www.gutenberg.org/files/28289/28289-0.txt"
auth4_url2 = "https://www.gutenberg.org/cache/epub/17780/pg17780.txt"
auth4_url3 = "https://www.gutenberg.org/files/550/550-0.txt"
auth4_url4 = "https://www.gutenberg.org/cache/epub/47025/pg47025.txt"
auth4_url5 = "https://www.gutenberg.org/files/10762/10762.txt"

In [291]:
auth5_url1 = "https://www.gutenberg.org/cache/epub/60278/pg60278.txt"
auth5_url2 = "https://www.gutenberg.org/cache/epub/66837/pg66837.txt"
auth5_url3 = "https://www.gutenberg.org/files/60327/60327-0.txt"
auth5_url4 = "https://www.gutenberg.org/files/3474/3474-0.txt"
auth5_url5 = "https://www.gutenberg.org/cache/epub/27180/pg27180.txt"

In [295]:
def extract_fragments(url, L):
    #Read the file
    file = urllib.request.urlopen(url)
    lines = []
    for line in file :
        decoded_line = line.decode("utf-8")
        lines.append(decoded_line.strip())
    
    #Change the letters to lower case
    raw_docs = [doc.lower() for doc in lines]
    while ("" in raw_docs) :
        raw_docs.remove("")
    
    #Word tokenization
    tokenized_doc = [word_tokenize(doc) for doc in raw_docs]
    
    #Remove the punctuations
    regex = re.compile('[%s]'% re.escape(string.punctuation))
    tokenized_docs_no_punc = []
    for review in tokenized_doc :
        new_review = []
        for token in review :
            new_token = regex.sub(u'', token)
            if not new_token == u'' :
                new_review.append(new_token)
        tokenized_docs_no_punc.append(new_review)
    
    puncs = ["’", "‘", "“", "”"]
    for punc in puncs :
        for line in tokenized_docs_no_punc :
            while(punc in line) :
                line.remove(punc)
    
    #Remove the stop words
    tokenized_doc_no_sw = []
    for doc in tokenized_docs_no_punc :
        new_term_vector = []
        for word in doc :
            if not word in stopwords.words('english') :
                new_term_vector.append(word)
        tokenized_doc_no_sw.append(new_term_vector)
    
    lists = []
    for line in tokenized_doc_no_sw:
        lines = []
        for word in line:
            lines.append("".join(u for u in word if u not in ("—")))
        lists.append(lines)
    
    #Use the lemmantization to get the root words
    wordnet = WordNetLemmatizer()
    pre_docs = []
    for doc in lists :
        final_doc = []
        for word in doc :
            final_doc.append(wordnet.lemmatize(word))
        pre_docs.append(final_doc)
    
    num = [random.randint(200,len(lists)) for  n in range (0,3000) ]
    numbers = set(num)
    
    #Extracting L number of fragments in size of 200
    final_list = []
    for i in numbers :
        if len(final_list) < 200 :
            line = lists[i]
            if len(line) >= L:
                digits = [dig for dig in line if dig.isdigit()]
                if len(digits) == 0:
                    final_list.append(' '.join(line[0:L]))
        else :
            break
    return final_list

In [74]:
L = 5

In [75]:
auth1_list1 = extract_fragments(auth1_url1, L)
auth1_list2 = extract_fragments(auth1_url2, L)
auth1_list3 = extract_fragments(auth1_url3, L)
auth1_list4 = extract_fragments(auth1_url4, L)
auth1_list5 = extract_fragments(auth1_url5, L)

In [76]:
list1 = [*auth1_list1, *auth1_list2, *auth1_list3, *auth1_list4, *auth1_list5]

In [77]:
auth2_list1 = extract_fragments(auth2_url1, L)
auth2_list2 = extract_fragments(auth2_url2, L)
auth2_list3 = extract_fragments(auth2_url3, L)
auth2_list4 = extract_fragments(auth2_url4, L)
auth2_list5 = extract_fragments(auth2_url5, L)

In [78]:
list2 = [*auth2_list1, *auth2_list2, *auth2_list3, *auth2_list4, *auth2_list5]

In [79]:
auth3_list1 = extract_fragments(auth3_url1, L)
auth3_list2 = extract_fragments(auth3_url2, L)
auth3_list3 = extract_fragments(auth3_url3, L)
auth3_list4 = extract_fragments(auth3_url4, L)
auth3_list5 = extract_fragments(auth3_url5, L)

In [80]:
list3 = [*auth3_list1, *auth3_list2, *auth3_list3, *auth3_list4, *auth3_list5]

In [81]:
auth4_list1 = extract_fragments(auth4_url1, L)
auth4_list2 = extract_fragments(auth4_url2, L)
auth4_list3 = extract_fragments(auth4_url3, L)
auth4_list4 = extract_fragments(auth4_url4, L)
auth4_list5 = extract_fragments(auth4_url5, L)

In [82]:
list4 = [*auth4_list1, *auth4_list2, *auth4_list3, *auth4_list4, *auth4_list5]

In [83]:
auth5_list1 = extract_fragments(auth5_url1, L)
auth5_list2 = extract_fragments(auth5_url2, L)
auth5_list3 = extract_fragments(auth5_url3, L)
auth5_list4 = extract_fragments(auth5_url4, L)
auth5_list5 = extract_fragments(auth5_url5, L)

In [84]:
list5 = [*auth5_list1, *auth5_list2, *auth5_list3, *auth5_list4, *auth5_list5]

In [100]:
x = [*list1, *list2, *list3, *list4, *list5]

In [101]:
len(x)

5000

In [102]:
x

['singing though sunday pleasant scents',
 'way coketown showed black mist',
 'mouths lean old horses worn',
 'space turn great wheel earth',
 'tracks however slight mounds grass',
 'distant long time solitude remained',
 'still rachael way untrodden think',
 'poor lad poor lad made',
 'rachael caught arms scream resounded',
 'terrific screams could got rachael',
 'sound twenty thirty times took',
 'wide prospect beautiful stillness minutes',
 'almost carried despair brave heart',
 'go different directions seeking aid',
 'one happened think stephen think',
 'turned went upon search stopped',
 'shawl guide place threw bonnet',
 'men lay shade asleep straw',
 'men drunken slumber comrade shouting',
 'message louisa wrote gave time',
 'village windlasses ropes poles candles',
 'things necessary fast collecting brought',
 'seemed hours hours since left',
 'men called listened done examined',
 'arose surface sat upon grass',
 'surgeon brought wine medicines expectation',
 'men keep besides 

In [90]:
with open("dataset(l=5).txt", 'w') as output:
    for row in x :
        output.write(str(row) + '\n')

In [89]:
x = []
filename = 'dataset(l=5).txt'
with open(filename) as file:
    while (line := file.readline().rstrip()):
        x.append(line)

In [92]:
authors = ['Charles Dickens', 'Jane Austen', 'Sir Arthur Conan Doyle', 'George Eliot', 'Mary Shelley']
target = []
for author in authors :
    for i in range(0, 1000):
        target.append(author)
y = LabelEncoder().fit_transform(target)

In [93]:
len(y)

5000

In [95]:
set(y)

{0, 1, 2, 3, 4}

## BOW

In [245]:
bow = CountVectorizer()

In [246]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_bow = bow.fit_transform(x_train)

Logistic Regression

In [247]:
lr_par = {
    'penalty' : ['l2'],
    'C' : [0.1, 1.0, 10.0],
    'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'max_iter' : [100, 200, 300]
}

In [248]:
LR = LogisticRegression()
lr_grid = GridSearchCV(LR, lr_par, refit = True, n_jobs=-1, cv=5)
lr_grid.fit(x_train_bow, y_train)
opt_par = lr_grid.best_params_
opt_par

{'C': 1.0, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}

In [298]:
LR = LogisticRegression(C = 1.0, max_iter = 100, penalty = 'l2', solver = 'newton-cg')

In [299]:
lr_train_acc = []
lr_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    LR.fit(x_train_bow, y_train)
    y_pred = LR.predict(x_test_bow)
    y_train_pred = LR.predict(x_train_bow)
    lr_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    lr_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [300]:
lr_mean_train_acc_bow = sum(lr_train_acc)/len(lr_train_acc)
lr_mean_test_acc_bow = sum(lr_test_acc)/len(lr_test_acc)

In [301]:
lr_mean_train_acc_bow, lr_mean_test_acc_bow

(0.9793125, 0.4800999999999999)

Support Vector Machines

In [249]:
svm_par = [
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['linear']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['poly'],
    'degree' : [2, 3, 4],
    'gamma' : ['scale', 'auto']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['rbf', 'sigmoid'],
    'gamma' : ['scale', 'auto']
    }
]

In [250]:
SVM = SVC()
svm_grid = GridSearchCV(SVM, svm_par, refit = True, n_jobs=-1, cv=5)
svm_grid.fit(x_train_bow, y_train)
opt_par = svm_grid.best_params_
opt_par

{'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}

In [302]:
SVM = SVC(C = 10.0, gamma = 'scale', kernel = 'rbf' )

In [303]:
svm_train_acc = []
svm_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    SVM.fit(x_train_bow, y_train)
    y_pred = SVM.predict(x_test_bow)
    y_train_pred = SVM.predict(x_train_bow)
    svm_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    svm_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [304]:
svm_mean_train_acc_bow = sum(svm_train_acc)/len(svm_train_acc)
svm_mean_test_acc_bow = sum(svm_test_acc)/len(svm_test_acc)

In [305]:
svm_mean_train_acc_bow, svm_mean_test_acc_bow

(1.0, 0.45135000000000003)

Random Forest

In [251]:
rf_par = {
    'n_estimators' : [50, 100, 200],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [8, 16, 32],
    'max_features' : ['sqrt', 'log2']
}

In [252]:
RF = RandomForestClassifier()
rf_grid = GridSearchCV(RF, rf_par, refit = True, n_jobs=-1, cv=5)
rf_grid.fit(x_train_bow, y_train)
opt_par = rf_grid.best_params_
opt_par

{'criterion': 'entropy',
 'max_depth': 32,
 'max_features': 'log2',
 'n_estimators': 200}

In [306]:
RF = RandomForestClassifier(criterion = 'entropy', max_depth = 32, max_features = 'log2', n_estimators = 200 )

In [307]:
rf_train_acc = []
rf_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    RF.fit(x_train_bow, y_train)
    y_pred = RF.predict(x_test_bow)
    y_train_pred = RF.predict(x_train_bow)
    rf_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    rf_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [308]:
rf_mean_train_acc_bow = sum(rf_train_acc)/len(rf_train_acc)
rf_mean_test_acc_bow = sum(rf_test_acc)/len(rf_test_acc)

In [309]:
rf_mean_train_acc_bow, rf_mean_test_acc_bow

(0.9527875, 0.4276000000000001)

Naive Bays 

In [310]:
NB = MultinomialNB()

In [311]:
nb_train_acc = []
nb_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    NB.fit(x_train_bow, y_train)
    y_pred = NB.predict(x_test_bow)
    y_train_pred = NB.predict(x_train_bow)
    nb_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    nb_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [312]:
nb_mean_train_acc_bow = sum(nb_train_acc)/len(nb_train_acc)
nb_mean_test_acc_bow = sum(nb_test_acc)/len(nb_test_acc)

In [313]:
nb_mean_train_acc_bow, nb_mean_test_acc_bow

(0.9110250000000001, 0.4972499999999999)

XGBoost

In [253]:
xgb_par = {
    'n_estimators' : [20, 50, 100],
    'use_label_encoder' : [False],
    'max_depth' : [8, 16, 32],
    'learning_rate' : [0.01, 0.1]
}

In [254]:
XGB = XGBClassifier()
xgb_grid = GridSearchCV(XGB, xgb_par, refit = True, n_jobs=-1, cv=5)
xgb_grid.fit(x_train_bow, y_train)
opt_par = xgb_grid.best_params_
opt_par

[23:09:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


{'learning_rate': 0.1,
 'max_depth': 32,
 'n_estimators': 100,
 'use_label_encoder': False}

In [314]:
XGB = XGBClassifier(learning_rate = 0.1, max_depth = 32, n_estimators = 100, use_label_encoder = False )

In [315]:
xgb_train_acc = []
xgb_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    XGB.fit(x_train_bow, y_train)
    y_pred = XGB.predict(x_test_bow)
    y_train_pred = XGB.predict(x_train_bow)
    xgb_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    xgb_test_acc.append(metrics.accuracy_score(y_test, y_pred))

[23:55:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:55:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:56:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:56:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

In [316]:
xgb_mean_train_acc_bow = sum(xgb_train_acc)/len(xgb_train_acc)
xgb_mean_test_acc_bow = sum(xgb_test_acc)/len(xgb_test_acc)

In [317]:
xgb_mean_train_acc_bow, xgb_mean_test_acc_bow

(0.7070125, 0.4375)

In [318]:
algorithms = ['Logistic Regression', 'Support Vector Machine', 'Random Forest', 'Naive Bayes', 'XGBoost']

In [319]:
bow_df = pd.DataFrame({
    'Algorithm' : [], 'Mean Train Accuracy' : [], 'Mean Test Accuracy' :[]
})

In [320]:
mean_train_acc_bow = [lr_mean_train_acc_bow, svm_mean_train_acc_bow, rf_mean_train_acc_bow,
                      nb_mean_train_acc_bow, xgb_mean_train_acc_bow]
mean_test_acc_bow = [lr_mean_test_acc_bow, svm_mean_test_acc_bow, rf_mean_test_acc_bow,
                    nb_mean_test_acc_bow, xgb_mean_test_acc_bow]

In [321]:
for i in range (0, len(algorithms)) :
    bow_df = bow_df.append({
        'Algorithm' : algorithms[i],
        'Mean Train Accuracy' : mean_train_acc_bow[i],
        'Mean Test Accuracy' : mean_test_acc_bow[i],
    }, ignore_index = True
    )

In [322]:
bow_df

,Algorithm,Mean Train Accuracy,Mean Test Accuracy
0,Logistic Regression,0.979313,0.48010
1,Support Vector Machine,1.000000,0.45135
2,Random Forest,0.952788,0.42760
3,Naive Bayes,0.911025,0.49725
4,XGBoost,0.707013,0.43750


In [323]:
bow_df.to_csv('BOW_Results.csv')

## Tf-idf

In [259]:
tf_idf = TfidfVectorizer(use_idf=True)

In [260]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_tf = tf_idf.fit_transform(x_train)

Logistic Regression

In [261]:
lr_par = {
    'penalty' : ['l2'],
    'C' : [0.1, 1.0, 10.0],
    'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'max_iter' : [100, 200, 300]
}

In [263]:
LR = LogisticRegression()
lr_grid = GridSearchCV(LR, lr_par, refit = True, n_jobs=-1, cv=5)
lr_grid.fit(x_train_tf, y_train)
opt_par = lr_grid.best_params_
opt_par

{'C': 1.0, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}

In [324]:
LR = LogisticRegression(C = 1.0, max_iter = 100, penalty = 'l2', solver = 'newton-cg')

In [325]:
lr_train_acc = []
lr_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    LR.fit(x_train_tf, y_train)
    y_pred = LR.predict(x_test_tf)
    y_train_pred = LR.predict(x_train_tf)
    lr_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    lr_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [326]:
lr_mean_train_acc_tf = sum(lr_train_acc)/len(lr_train_acc)
lr_mean_test_acc_tf = sum(lr_test_acc)/len(lr_test_acc)

In [327]:
lr_mean_train_acc_tf, lr_mean_test_acc_tf

(0.9215875, 0.4869)

Support Vector Machine

In [265]:
svm_par = [
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['linear']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['poly'],
    'degree' : [2, 3, 4],
    'gamma' : ['scale', 'auto']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['rbf', 'sigmoid'],
    'gamma' : ['scale', 'auto']
    }
]

In [266]:
SVM = SVC()
svm_grid = GridSearchCV(SVM, svm_par, refit = True, n_jobs=-1, cv=5)
svm_grid.fit(x_train_tf, y_train)
opt_par = svm_grid.best_params_
opt_par

{'C': 1.0, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}

In [328]:
SVM = SVC(C = 1.0, degree = 2, gamma = 'scale', kernel = 'poly' )

In [329]:
svm_train_acc = []
svm_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    SVM.fit(x_train_tf, y_train)
    y_pred = SVM.predict(x_test_tf)
    y_train_pred = SVM.predict(x_train_tf)
    svm_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    svm_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [330]:
svm_mean_train_acc_tf = sum(svm_train_acc)/len(svm_train_acc)
svm_mean_test_acc_tf = sum(svm_test_acc)/len(svm_test_acc)

In [331]:
svm_mean_train_acc_tf, svm_mean_test_acc_tf

(0.9997499999999999, 0.4967999999999999)

Random Forest

In [267]:
rf_par = {
    'n_estimators' : [50, 100, 200],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [8, 16, 32],
    'max_features' : ['sqrt', 'log2']
}

In [268]:
RF = RandomForestClassifier()
rf_grid = GridSearchCV(RF, rf_par, refit = True, n_jobs=-1, cv=5)
rf_grid.fit(x_train_tf, y_train)
opt_par = rf_grid.best_params_
opt_par

{'criterion': 'gini',
 'max_depth': 32,
 'max_features': 'log2',
 'n_estimators': 200}

In [332]:
RF = RandomForestClassifier(criterion = 'gini', max_depth = 32, max_features = 'log2', n_estimators = 200)

In [333]:
rf_train_acc = []
rf_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    RF.fit(x_train_tf, y_train)
    y_pred = RF.predict(x_test_tf)
    y_train_pred = RF.predict(x_train_tf)
    rf_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    rf_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [334]:
rf_mean_train_acc_tf = sum(rf_train_acc)/len(rf_train_acc)
rf_mean_test_acc_tf = sum(rf_test_acc)/len(rf_test_acc)

In [335]:
rf_mean_train_acc_tf, rf_mean_test_acc_tf

(0.8920000000000001, 0.41744999999999993)

Naive Bayes

In [336]:
NB = MultinomialNB()

In [337]:
nb_train_acc = []
nb_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    NB.fit(x_train_tf, y_train)
    y_pred = NB.predict(x_test_tf)
    y_train_pred = NB.predict(x_train_tf)
    nb_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    nb_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [338]:
nb_mean_train_acc_tf = sum(nb_train_acc)/len(nb_train_acc)
nb_mean_test_acc_tf = sum(nb_test_acc)/len(nb_test_acc)

In [339]:
nb_mean_train_acc_tf, nb_mean_test_acc_tf

(0.917225, 0.49985000000000007)

XGBoost

In [269]:
xgb_par = {
    'n_estimators' : [20, 50, 100],
    'use_label_encoder' : [False],
    'max_depth' : [8, 16, 32],
    'learning_rate' : [0.01, 0.1]
}

In [270]:
XGB = XGBClassifier()
xgb_grid = GridSearchCV(XGB, xgb_par, refit = True, n_jobs=-1, cv=5)
xgb_grid.fit(x_train_tf, y_train)
opt_par = xgb_grid.best_params_
opt_par

[23:26:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


{'learning_rate': 0.1,
 'max_depth': 32,
 'n_estimators': 100,
 'use_label_encoder': False}

In [340]:
XGB = XGBClassifier(learning_rate = 0.1, max_depth = 32, n_estimators = 100, use_label_encoder = False)

In [341]:
xgb_train_acc = []
xgb_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    XGB.fit(x_train_tf, y_train)
    y_pred = XGB.predict(x_test_tf)
    y_train_pred = XGB.predict(x_train_tf)
    xgb_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    xgb_test_acc.append(metrics.accuracy_score(y_test, y_pred))

[00:01:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:02:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:02:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:02:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

In [342]:
xgb_mean_train_acc_tf = sum(xgb_train_acc)/len(xgb_train_acc)
xgb_mean_test_acc_tf = sum(xgb_test_acc)/len(xgb_test_acc)

In [343]:
xgb_mean_train_acc_tf, xgb_mean_test_acc_tf

(0.8508375000000001, 0.3986)

In [344]:
algorithms = ['Logistic Regression', 'Support Vector Machine', 'Random Forest', 'Naive Bayes', 'XGBoost']

In [345]:
tf_df = pd.DataFrame({
    'Algorithm' : [], 'Mean Train Accuracy' : [], 'Mean Test Accuracy' :[]
})

In [346]:
mean_train_acc_tf = [lr_mean_train_acc_tf, svm_mean_train_acc_tf, rf_mean_train_acc_tf,
                      nb_mean_train_acc_tf, xgb_mean_train_acc_tf]
mean_test_acc_tf = [lr_mean_test_acc_tf, svm_mean_test_acc_tf, rf_mean_test_acc_tf,
                    nb_mean_test_acc_tf, xgb_mean_test_acc_tf]

In [347]:
for i in range (0, len(algorithms)) :
    tf_df = tf_df.append({
        'Algorithm' : algorithms[i],
        'Mean Train Accuracy' : mean_train_acc_tf[i],
        'Mean Test Accuracy' : mean_test_acc_tf[i],
    }, ignore_index = True
    )

In [348]:
tf_df

,Algorithm,Mean Train Accuracy,Mean Test Accuracy
0,Logistic Regression,0.921588,0.48690
1,Support Vector Machine,0.999750,0.49680
2,Random Forest,0.892000,0.41745
3,Naive Bayes,0.917225,0.49985
4,XGBoost,0.850838,0.39860


In [349]:
tf_df.to_csv('TF-IDF_Results.csv')